In [2]:
from astroquery.vizier import Vizier
import pyvo

## Get entire table with Astroquery

First find out about the VizieR ID of your table of interest. 

In [3]:
catalog_list = Vizier.find_catalogs('Jenkins 2009')
for k, v in catalog_list.items():
    print(k, ': ', v.description)

J/ApJ/700/1299 :  Gas-phase element depletions in the ISM (Jenkins, 2009)
J/ApJ/704/975 :  Rotational velocities for M dwarfs (Jenkins+, 2009)
J/ApJS/176/59 :  FUSE survey of OVI in the disk of the Milky Way (Bowen+, 2008)
J/ApJS/177/39 :  Survey of low-redshift OVI absorbers (Tripp+, 2008)
J/PASJ/52/997 :  VSOP 5 GHz AGN Survey (Hirabayashi+, 2000)


Now we know that we are after the catalogue called `J/ApJ/700/1299`. If you would now do `get_catalogs` you'd only get 50 rows in each table. So we first set the row limit to infinity (a bit angerous with things like Gaia but fine for our purpose). Then we download the entire catalogue made up of 7 tables. For more details on restricting for certain columns, or certain values, see here: https://astroquery.readthedocs.io/en/latest/vizier/vizier.html

In [6]:
Vizier.ROW_LIMIT = -1
catalog = Vizier.get_catalogs('J/ApJ/700/1299')
catalog

TableList with 7 tables:
	'0:J/ApJ/700/1299/table4' with 15 column(s) and 16 row(s) 
	'1:J/ApJ/700/1299/table2' with 15 column(s) and 276 row(s) 
	'2:J/ApJ/700/1299/table5' with 14 column(s) and 239 row(s) 
	'3:J/ApJ/700/1299/table6' with 15 column(s) and 29 row(s) 
	'4:J/ApJ/700/1299/table7' with 12 column(s) and 1475 row(s) 
	'5:J/ApJ/700/1299/refs' with 4 column(s) and 140 row(s) 
	'6:J/ApJ/700/1299/notes' with 2 column(s) and 44 row(s) 

We got a list of tables, to get table5, we just pick the third entry. It is immediately an astropy Table object and you can handle it like any astrop Table

In [7]:
table5 = catalog[2]
print(type(table5))
table5

<class 'astropy.table.table.Table'>


<Table length=239>
  HD   Vcomp      Name     b_logNHo ...  e_F_s    Prob             El          
       km / s               [cm-2]  ...                                        
int32  int16    bytes16    float32  ... float32 float32         bytes41        
------ ------ ------------ -------- ... ------- ------- -----------------------
  1383     --      HD 1383    21.43 ...    0.08   0.510        O Mg Mn Ni Cu Ge
  2905     --  {kappa} Cas    21.17 ...    0.14   0.989    Mg Cl Ti Mn Fe Ni Cu
  5394     --  {gamma} Cas    20.06 ...    0.05   0.306    N O Mg P Cl Ti Mn Fe
 12323     --     HD 12323    21.21 ...    0.07   0.011     O Mg Mn Fe Ni Cu Ge
 13268     --     HD 13268    21.35 ...    0.08   0.147        O Mg Mn Ni Cu Ge
 13745     --     HD 13745    21.30 ...      --      --                   Mg Fe
 14434     --     HD 14434    21.40 ...    0.09   0.301           O Mg Mn Ni Cu
 15137     --     HD 15137    21.10 ...      --      --                   Mg Fe
 18100     --     HD 18100    20.02 ...    0.08   0.000  Mg Si P Cr Mn Fe Ni Zn
   ...    ...          ...      ... ...     ...     ...                     ...
220057     --    HD 220057    21.02 ...    0.06   0.131 N O Mg P Mn Ni Cu Ge Kr
224151     --     V373 Cas    21.40 ...    0.05   0.666          N O Mg P Fe Kr
224572     --  {sigma} Cas    20.87 ...    0.40   0.559        Mg P Cl Mn Fe Cu
232522     --   HDE 232522    21.14 ...    0.08   0.797           O Mg Mn Ni Ge
303308     --   HDE 303308    21.41 ...    0.05   0.169           O Mg Ti Fe Kr
308813     --   HDE 308813    21.21 ...    0.08   0.679        O Mg Mn Ni Cu Ge
    --     --  BD +35 4258    21.20 ...      --      --                   Mg Fe
    --     --  BD +53 2820    21.31 ...    0.09   0.204           O Mn Ni Cu Ge
    --     -- CPD -59 2603    21.11 ...      --      --                   Mg Fe
    --     -- CPD -69 1743    21.08 ...    0.09   0.113           O Mn Ni Cu Ge

## Get entire table with pyVO
This is a second, more general package that allows to access all Virtual Observatory compliant services with ADQL queries (if you haven't hear of it, it's like SQL but optimised for astronomy).

First we need to tell it where we can access the VizieR TAP (Table access protocol) point. Then we ask for all tables in VizieR that have a name similar to the one, we are looking for. Note that here we are looking at individual tables rather than catalogs. So table5 would have its own entry. The `%` symbols are wildcards. 

In [18]:
tap_vizier = pyvo.dal.TAPService('http://tapvizier.u-strasbg.fr/TAPVizieR/tap')
mass_psc_set = tap_vizier.search("SELECT  *  FROM tables " + 
                                 "WHERE description LIKE '%Jenkins%' " + 
                                 "AND table_name LIKE '%table5%'").to_table()
mass_psc_set['table_name', 'description']

<Table length=4>
      table_name      ...
       bytes36        ...
--------------------- ...
J/A+A/621/A112/table5 ...
J/ApJ/700/1299/table5 ...
 J/ApJ/774/136/table5 ...
 J/ApJ/636/631/table5 ...

Now we know that we are after `J/ApJ/700/1299/table5` and we can download the entire table as follows. You could restrict your query to say H column densities above 10$^21$cm$^{-2}$

In [10]:
table5 = tap_vizier.search("SELECT * FROM \"J/ApJ/700/1299/table5\" ").to_table()
print(type(table5))
table5

<class 'astropy.table.table.Table'>


<Table length=239>
recno   HD   Vcomp      Name     ...  e_F*s    Prob             El          
             km / s              ...                                        
int32 int32  int16     object    ... float64 float64          object        
----- ------ ------ ------------ ... ------- ------- -----------------------
    1   1383      0      HD 1383 ...    0.08    0.51        O Mg Mn Ni Cu Ge
    2   2905      0  {kappa} Cas ...    0.14   0.989    Mg Cl Ti Mn Fe Ni Cu
    3   5394      0  {gamma} Cas ...    0.05   0.306    N O Mg P Cl Ti Mn Fe
    4  12323      0     HD 12323 ...    0.07   0.011     O Mg Mn Fe Ni Cu Ge
    5  13268      0     HD 13268 ...    0.08   0.147        O Mg Mn Ni Cu Ge
    6  13745      0     HD 13745 ...      --      --                   Mg Fe
    7  14434      0     HD 14434 ...    0.09   0.301           O Mg Mn Ni Cu
    8  15137      0     HD 15137 ...      --      --                   Mg Fe
    9  18100      0     HD 18100 ...    0.08     0.0  Mg Si P Cr Mn Fe Ni Zn
  ...    ...    ...          ... ...     ...     ...                     ...
  230 220057      0    HD 220057 ...    0.06   0.131 N O Mg P Mn Ni Cu Ge Kr
  231 224151      0     V373 Cas ...    0.05   0.666          N O Mg P Fe Kr
  232 224572      0  {sigma} Cas ...     0.4   0.559        Mg P Cl Mn Fe Cu
  233 232522      0   HDE 232522 ...    0.08   0.797           O Mg Mn Ni Ge
  234 303308      0   HDE 303308 ...    0.05   0.169           O Mg Ti Fe Kr
  235 308813      0   HDE 308813 ...    0.08   0.679        O Mg Mn Ni Cu Ge
  236      0      0  BD +35 4258 ...      --      --                   Mg Fe
  237      0      0  BD +53 2820 ...    0.09   0.204           O Mn Ni Cu Ge
  238      0      0 CPD -59 2603 ...      --      --                   Mg Fe
  239      0      0 CPD -69 1743 ...    0.09   0.113           O Mn Ni Cu Ge